# 문의, 추천 질문 말뭉치 완성형으로 만들어보기

- 현재까지 만든 문장 형태는 크게 문의(장소 / 리뷰나 후기 / 가격 문의) / 추천 으로 어미까지만 만들어 둔 상태
- 문장을 형용사 및 명사(장소) 등을 추가하여 완성형 문장으로 만들고 이를 학습시키도록 하자.
- 첫번쨰로 사용자의 질문의 의도를 우선 파악하여 문의인지 추천을 요구하는지를 인지한다. 문의 관련 질문일 경우 1) 장소 / 2)가격 / 3)리뷰,후기 문의 로 총 3가지로 한다. 추천 관련 질문일 경우 총 6가지 항목을 기반으로 진행, 1) 청결 / 2) 서비스(고객 만족도) / 3) 매장 분위기,인테리어 / 4) 가격 / 5) 위치/교통 , 6)헤어 관련 만족도(디자이너 실력, 시술 후 헤어 상태 등 헤어 관련 키워드) 로 해당 항목 기준을 기반하여 추천한다.

## 문의 (1.장소 / 2.리뷰나 후기 / 3.가격 문의) 질문 말뭉치 
- 가게상호명과 문의 어미를 결합, 완성형 문장을 만든다
- ex) 강남역 준오헤어 어디에 있나요? / 강남역 준오헤어 2호점 솔직 리뷰 궁금해요.
- ex) 강남역 준오헤어 2호점 남자 커트 얼마죠?


## 추천( 1.청결 / 2.서비스(고객 만족도) / 3.매장 분위기,인테리어 / 4.가격 / 5.위치/교통 , 6. 헤어 관련 만족도(디자이너 실력, 시술 후 헤어 상태 등 헤어 관련 키워드) 질문 말뭉치

- 형용사+가게 상호명+추천 어미 결합, 완성형 문장을 만든다.
- ex) 강남역 근처에 가까운 남자 컷트 하는 미용실 어디에 있어요? 요즘 후기 제일 좋은 미용실 강남역 근처로 추천해주세요. 펌 잘하는 디자이너 있는 미용실 강남에 있어요? 등 

In [631]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import re
from konlpy.tag import Mecab
import platform
path = "c:/Windows/Fonts/malgun.ttf" 
from matplotlib import font_manager, rc 
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') 
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

In [632]:
# 질문어미 앞에 추가해서 붙일 형용사 질문 데이터셋 불러오기
df_adj = pd.read_excel('./문의_추천 용 질문을 위한 형용사 표현.xlsx'
                       , encoding= 'utf-8-sig')
df_adj.head()

,san_pos_keywords,cs_pos_keywords,mood_pos_keywords,price_pos_keywords,loc_pos_keywords,sat_pos_keywords,etc_keywords
0,깔끔한,친절한,분위기가 좋은,싼,가까운,인스타각의,결혼식용으로 할
1,깨끗한,불친절하지 않은,화사한,가격이 저렴한,가까운 곳인,그뤠잇한,상견례용으로 할
2,청결한,친절한,넓은,혜자스러운,위치가 좋은,개쩌는,입학식용으로 할
3,위생적인,서비스가 좋은,넓직한,가격이 착한,접근성이 좋은,대만족의,졸업식때 할
4,빛나는,서비스 잘해주는,모던한,착한 가격의,금방 걸어갈수 있는,만족할만한,입학식때 할


In [633]:
df_adj.tail() #컬럼별 데이터 갯수 상이하기에 일치시켜주던지 nan값 삭제해줘야함

,san_pos_keywords,cs_pos_keywords,mood_pos_keywords,price_pos_keywords,loc_pos_keywords,sat_pos_keywords,etc_keywords
88,NaN,NaN,NaN,NaN,NaN,얼굴에 어울리는,NaN
89,NaN,NaN,NaN,NaN,NaN,아름다워 보이는,NaN
90,NaN,NaN,NaN,NaN,NaN,이쁜,NaN
91,NaN,NaN,NaN,NaN,NaN,두상에 맞는,NaN
92,NaN,NaN,NaN,NaN,NaN,머리 빨리해주는,NaN


In [634]:
san_adjs = df_adj['san_pos_keywords'].dropna().tolist()
cs_adjs = df_adj['cs_pos_keywords'].dropna().tolist()
mood_adjs = df_adj['mood_pos_keywords'].dropna().tolist()
price_adjs = df_adj['price_pos_keywords'].dropna().tolist()
loc_adjs = df_adj['loc_pos_keywords'].dropna().tolist()
sat_adjs = df_adj['sat_pos_keywords'].dropna().tolist()
etc_adjs = df_adj['etc_keywords'].dropna().tolist()

In [635]:
len(san_adjs),len(cs_adjs),len(mood_adjs),len(price_adjs),len(loc_adjs),len(sat_adjs),len(etc_adjs)

(20, 20, 32, 26, 15, 93, 50)

In [636]:
#혹시 모를 형용사 접두사에 중복이 있는지 확인 후 제거해서 list로 다시 저장
san_adjs = list(set(san_adjs))
cs_adjs = list(set(cs_adjs))
mood_adjs = list(set(mood_adjs))
price_adjs = list(set(price_adjs))
loc_adjs = list(set(loc_adjs))
sat_adjs = list(set(sat_adjs))
etc_adjs = list(set(etc_adjs))

In [637]:
total_adjs = san_adjs+cs_adjs+mood_adjs+price_adjs+loc_adjs+sat_adjs+etc_adjs
len(total_adjs) #중복제외된 형용사 접두 어미 245개

245

In [638]:
total_adjs_df = pd.DataFrame({
    'total_adjs' : total_adjs
})

In [639]:
total_adjs_df[190:210]

,total_adjs
190,얼굴선을 가려주는
191,머리잘하기로 유명한
192,트위터서 유명한
193,그뤠잇한
194,인스타각의
195,잘하기로 소문난
196,가을에 맞는
197,졸업식때 할
198,겨울맞이용
199,가을에 할만한


In [640]:
total_adjs_df.loc[:19, 'main'] ='sanitation'
total_adjs_df.loc[19:38, 'main'] = 'customer_service'
total_adjs_df.loc[38:69, 'main'] = 'mood'
total_adjs_df.loc[69:93, 'main'] = 'price'
total_adjs_df.loc[93:108, 'main'] = 'location'
total_adjs_df.loc[108:196, 'main'] = 'satisfaction'
total_adjs_df.loc[196:255, 'main'] = 'etc'

## 문의_추천용 질문을 위한~~.xlsx 파일에 혹시 질문어미 더 추가해 주셨나요? 
## 그렇다면 df에 인덱스 수정이 들어가야 하는데 해당건 찍어서 확인 꼭 해주시고 df의 main 컬럼 부분 제대로 분류 되었는지 csv 파일 열어서 본인이 직접 확인 꼭! 하세요!! 
- 작성된 df는 xlsx 파일 작성일 기준이 2020-05-25, 오후 3시 30분 입니다.

In [641]:
total_adjs_df[150:200] 
#이런식으로 df의 main 부분이 적합하게 분류되었는지 다 확인해 볼것!

,total_adjs,main
150,후기가 제일 많은,satisfaction
151,실력있는,satisfaction
152,대만족의,satisfaction
153,경력이 상당한,satisfaction
154,경력이 많은,satisfaction
155,금손인 쌤이 많은,satisfaction
156,유명한,satisfaction
157,금손인 디자이너가 많은,satisfaction
158,늘 만족스러운,satisfaction
159,머리잘하는,satisfaction


## 상호명 데이터 불러와 질문 셋 구체화하기
- 시는 모두 서울이기 때문에 구분하지 않아도, 구와 동까지 구분하는 컬럼이 들어가질문을 작성하게 된다면 위치 정보를 보다 상세하게 나눠야 겠으나 우선은 샘플 데이터 기반에 상호명만 가져다 써서 질문 데이터셋을 만듬

In [642]:
df_hairshop = pd.read_csv('./HairshopReviewsFinal_2020-05-28.csv'
            , encoding='utf-8-sig'
           , index_col=0)
df_hairshop.tail()
#해당 csv는 전처리가 완료된 최종 데이터 tel에 존재, nearby_station_distance 에도 nan 존재

,store_id,store_name,tel,review,addrRoad,district,neighborhood,coord_x,coord_y,nearby_station,nearby_station_distance,type,collect_day
817240,1257948890,프로젝트라빠헤어 신촌점,02-322-1767,진짜 분위기도 너무 공주님 같고 디자이너 분들도 다들 친절하시고 위치도 찾기 어려운...,서울특별시 서대문구 연세로4길 1 3층,서대문구,"창천동,신촌동",126.937133,37.55707,"신촌역,서강대역,이대역,홍대입구역","208,565,822,885",kakao,2020-05-22
817241,1257948890,프로젝트라빠헤어 신촌점,02-322-1767,제 점수는요 두구두구~ ⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐...,서울특별시 서대문구 연세로4길 1 3층,서대문구,"창천동,신촌동",126.937133,37.55707,"신촌역,서강대역,이대역,홍대입구역","208,565,822,885",kakao,2020-05-22
817242,1257948890,프로젝트라빠헤어 신촌점,02-322-1767,정말추천합니다 이런거 리뷰 귀찮아서 잘 안쓰는데...,서울특별시 서대문구 연세로4길 1 3층,서대문구,"창천동,신촌동",126.937133,37.55707,"신촌역,서강대역,이대역,홍대입구역","208,565,822,885",kakao,2020-05-22
817243,1257948890,프로젝트라빠헤어 신촌점,02-322-1767,오늘 처음으로 방문했는데 정말 다들 너무 친절하시고 디자이너분 머리도 이쁘게 해주시...,서울특별시 서대문구 연세로4길 1 3층,서대문구,"창천동,신촌동",126.937133,37.55707,"신촌역,서강대역,이대역,홍대입구역","208,565,822,885",kakao,2020-05-22
817244,1257948890,프로젝트라빠헤어 신촌점,02-322-1767,빠르고 이쁘게 되서 만족합니다. 실제 자르는데 30분도 안걸린거같네요. 머리감는 시...,서울특별시 서대문구 연세로4길 1 3층,서대문구,"창천동,신촌동",126.937133,37.55707,"신촌역,서강대역,이대역,홍대입구역","208,565,822,885",kakao,2020-05-22


In [643]:
df_hairshop.shape

(817245, 13)

In [644]:
df_hairshop.store_name

0          준오헤어 노원사거리점
1          준오헤어 노원사거리점
2          준오헤어 노원사거리점
3          준오헤어 노원사거리점
4          준오헤어 노원사거리점
              ...     
817240    프로젝트라빠헤어 신촌점
817241    프로젝트라빠헤어 신촌점
817242    프로젝트라빠헤어 신촌점
817243    프로젝트라빠헤어 신촌점
817244    프로젝트라빠헤어 신촌점
Name: store_name, Length: 817245, dtype: object

In [645]:
df_store = df_hairshop['store_name'].unique().tolist()

In [646]:
len(df_store) 

2706

### 주의사항! .unique()를 써서 중복된 상호명을 삭제하는것은 동명의 상호를 삭제하는 것일수도 있기 때문에 나중에 질문-대답 코드 처리시 동일 상호명이 있는지 확인 필요함!

In [647]:
#unique 사용할 경우 df_store 내용 가지고 df 만들것 / 우선 df_store 로 적용함
df_store

['준오헤어 노원사거리점',
 '차홍룸 홍대점',
 '차홍룸 용산센트럴점',
 '차홍룸 신논현점',
 'CHOP헤어 신천점',
 '차홍룸 목동점',
 '차홍룸 신촌점',
 '차홍룸 한남점',
 '제오헤어 신촌점',
 'CHOP헤어 잠실점',
 '차홍룸 잠실점',
 '에이엠톤 본점',
 '유니크바이',
 '박승철헤어스투디오 신촌점',
 '박승철헤어스투디오 신림점',
 '준오헤어 서울대입구역점',
 '준오헤어 서울대역1호점',
 '메종드아이디헤어 CBS목동점',
 '리안헤어 중랑역점',
 '이진헤어 노량진점',
 'CHOPHAIR 대치점',
 '박준뷰티랩 종각역점',
 '준오헤어 강남2호점',
 '더 부띠끄 건대점',
 '차홍룸 대치점',
 '이철헤어커커 명동1호점',
 '준오헤어 방배역점',
 '준오헤어 방배역2호점',
 '준오헤어 노원역점',
 '그라티아',
 '박승철헤어스투디오 홍대점',
 '박승철헤어스튜디오 홍대점',
 '이철헤어커커 신림점',
 '더 부띠끄 강남점',
 '제오헤어 이수점',
 '쟝피엘헤어 서울대입구역점',
 '리안헤어 신내점',
 '준오헤어 이수역점',
 '파리지엔헤어 명동점',
 '파란헤어 강남본점',
 '보그헤어 신도림역점',
 '와이미용실',
 '헤어벨리시마 건대점',
 '리안헤어 문정법조타운점',
 '준오헤어 돈암1호점',
 '박승철헤어스투디오 신도림홈플러스점',
 '준오헤어 서울대샤로수길점',
 '준오헤어 서울대역2호점',
 '이철헤어커커 오목교역점',
 '더퍼스트헤어 안암본점',
 '헤어아티스트M 영등포점',
 '어반그라운드헤어 by다비 왕십리역점',
 '준오헤어 잠실점',
 '더헤어 감동',
 '준오헤어 왕십리역1호점',
 '박승철헤어스투디오 마곡나루역점',
 '어반그라운드헤어 약수점',
 '헤어살롱드보르 약수점',
 '코벨헤어 본점',
 '준오헤어 남부터미널역점',
 '리안헤어 미아사거리역점',
 '트리플헤어디자인',
 '12월애헤어',
 '라트리헤어 스파힐 강남베르점',
 '준오헤어 건대역2호점',
 '공간헤어',
 '리안

In [648]:
df_hairshop[5950:5960] #가장 가까운 역이 맨 앞에 있는 역 키워드임

,store_id,store_name,tel,review,addrRoad,district,neighborhood,coord_x,coord_y,nearby_station,nearby_station_distance,type,collect_day
5950,1571970153,차홍룸 홍대점,02-336-3656,제 얼굴에 맞는 스타일을 해주셔서 믿고 맡길수 있어요 친절하시고 머리도 이쁘게 잘 ...,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5951,1571970153,차홍룸 홍대점,02-336-3656,너무나 친절하시고 머리 손질도 너무 쉽게 잘 알려주셔서 너무 감사드려요 ㅠㅠ 다음에...,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5952,1571970153,차홍룸 홍대점,02-336-3656,최고의 staff과 실장님입니다 항상 이뿐머리 고운머리 감사해요,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5953,1571970153,차홍룸 홍대점,02-336-3656,배희진실장님 지난번에 염색하고 색깔이 너무 마음에 들어서 재방문했는데 이번에도 너무...,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5954,1571970153,차홍룸 홍대점,02-336-3656,컷트도 꼼꼼하세 해주시고 스타일링도 자세히 알랴주셨어요,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5955,1571970153,차홍룸 홍대점,02-336-3656,너무예뻐요 이제 미용실 여기로만 올거예요 커트를 너무 잘해주셧어요,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5956,1571970153,차홍룸 홍대점,02-336-3656,머리 예쁘게 해주셔서 감사합니다 매번 디자이너님이랑 스탭분 너무 친절하셔서 좋아요 ...,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5957,1571970153,차홍룸 홍대점,02-336-3656,플라워펌을 하러갔는데 머리가 탈색모가 남아있어서 걱정했지만 너무 예쁘게 잘 나왔어요...,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5958,1571970153,차홍룸 홍대점,02-336-3656,다른곳에서 한 머리가 맘에 안 들었었는데 커트랑 앞머리펌으로 살려주셨어요 감사합니다...,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22
5959,1571970153,차홍룸 홍대점,02-336-3656,머리도 항상 잘 해주시고 아로마로 간단 마사지도 해주셔서 좋아요,서울특별시 마포구 양화로 61,마포구,"서교동,서교동",126.915071,37.550821,"합정역,망원역,상수역","114,731,733",booking,2020-05-22


In [649]:
df_hair=df_hairshop[['store_name','district','neighborhood','nearby_station']].drop_duplicates()
df_hair.head()

,store_name,district,neighborhood,nearby_station
0,준오헤어 노원사거리점,노원구,"상계동,상계6.7동",노원역
5947,차홍룸 홍대점,마포구,"서교동,서교동","합정역,망원역,상수역"
10731,차홍룸 용산센트럴점,용산구,"한강로2가,한강로동","신용산역,용산역,삼각지역,이촌역"
16159,차홍룸 신논현점,서초구,"서초동,서초4동","신논현역,강남역,사평역,논현역,언주역"
21098,CHOP헤어 신천점,송파구,"잠실동,잠실본동","종합운동장역,잠실새내역,삼전역"


- hair_category_1과 _2 로 나눈다. _1은 대략적인 머리 질문(머리,컷,펌) _2는 구체적인 헤어스타일 이름명을 삽입.

- top_hairstyle 컬럼 만드려고 했음(한울님과 상의된 부분)
- 가장 많이 조회/언급된 헤어스타일이 가장 그 헤어샵에서 잘하는 헤어스타일이라고 가정했기 때문이다. 머리를 잘한다 못한다의 평가는 헤어서비스를 받는 고객의 절대적인 주관성이 들어나는 부분이라 더더욱 평가가 애매모호함 =>5.28일 기준 x

- 문제점: 단순히 많이 언급된 것이 그 헤어스타일을 잘 한다고 볼수는 없다. 이렇게 된다면 강남, 종로 등 번화가에 위치, 내방객이 많은 대형 헤어체인점과 같은곳이 가장 잘하는 곳이 되기 때문이다. -> 그렇다고 정보가 가장 많은 번화가의 대형 헤어샵 체인점이 잘못된 것은 아니다. 정보의 불균형(비대칭)은 우리가 어떻게 할수 있는 방법이 아님

In [650]:
rough_hair_style=['머리','컷','펌']

In [651]:
specific_hair_style=['댄디컷','리젠트컷','스왓컷','크롭컷','포마드컷','쉼표머리'
                ,'투블럭','모히칸','머쉬룸컷','레이어드컷','울프컷','언밸런스컷'
                ,'히메컷','보브컷','샤기컷','허쉬컷','처피뱅','러블리펌','C컬펌'
                ,'S컬펌','물결펌','히피펌','글램펌','트위스트펌','발롱펌'
                ,'베이비펌','바디펌','루즈펌','가르마펌','쉐도우펌','포마드펌'
                ,'스핀스왈로우 펌','애즈펌','리젠트펌','다운펌','드레드락','호일펌'
                ,'볼륨매직펌','매직펌','에어펌','내츄럴펌','바그펌','보니펌','레인펌'
                ,'프릴펌','페미닌펌','이지펌','아이롱펌','보울컷','리프컷','고데기펌'
                ,'웨이브펌','복구펌','젤리펌','볼륨매직','픽시컷','보브컷','언발란스컷'
                ,'가일컷','가일펌','그레이스펌','태슬컷','디자인펌','붙임머리','셋팅펌'
                ,'엘리자벳펌','리프펌','스핀스왈로펌','시스루뱅','투블럭펌']

In [652]:
len(specific_hair_style)

70

In [653]:
df_hair['hair_category1'] = ','.join(rough_hair_style)
df_hair['hair_category2'] = ','.join(specific_hair_style)

In [654]:
df_hair

,store_name,district,neighborhood,nearby_station,hair_category1,hair_category2
0,준오헤어 노원사거리점,노원구,"상계동,상계6.7동",노원역,"머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
5947,차홍룸 홍대점,마포구,"서교동,서교동","합정역,망원역,상수역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
10731,차홍룸 용산센트럴점,용산구,"한강로2가,한강로동","신용산역,용산역,삼각지역,이촌역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
16159,차홍룸 신논현점,서초구,"서초동,서초4동","신논현역,강남역,사평역,논현역,언주역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
21098,CHOP헤어 신천점,송파구,"잠실동,잠실본동","종합운동장역,잠실새내역,삼전역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
...,...,...,...,...,...,...
815495,페이지헤어,도봉구,"도봉동,도봉2동",방학역,"머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
815496,SM프로헤어 한국외대점,동대문구,"이문동,이문1동","외대앞역,회기역,신이문역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
816376,론헤어 그루밍점,강남구,"신사동,신사동","신사역,논현역,잠원역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
816837,살롱에이투지 헬리오시티점,송파구,"가락동,가락1동","송파역,석촌역,가락시장역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."


## 추천 질문 셋 만들기

In [655]:
#pickle 형태로 만들어둔 어미 데이터 불러오기
import pickle
with open('./req_list_dict.pickle','rb') as f:
    req_list_dict = pickle.load(f)

In [656]:
len(req_list_dict.keys()), max(req_list_dict.keys()) #총 원소 갯수는 12875개, 가장 큰 인덱스 12874개

(12875, 12874)

In [657]:
req_list_dict

{0: '최고로 잘하는 데 어디지',
 1: '하고싶은 데 알고계신가요',
 2: '최고인 곳 뭐지요',
 3: '걱정없는 곳 있는가요',
 4: '하고싶은 헤어샵 궁금한데요',
 5: '괜찮은 데 알려줄수있나요',
 6: '같이오고 싶은 데 있는지',
 7: '아는 미용실 어디에요',
 8: '하고싶은 데 어디지',
 9: '최고로 잘하는 곳 있는지',
 10: '데려오고 싶은 헤어샵 있으신거죠',
 11: '같이오고 싶은 미용실 있어',
 12: '추천가능한 곳 있는거죠',
 13: '최고로 잘하는 곳 질문합니다',
 14: '예쁘게 하는 미용실 궁금해요',
 15: '하고싶은 헤어샵 어데고',
 16: '잘하는 미용실 알아요',
 17: '오지는 데 있는가요',
 18: '실패한적 없는 헤어샵 있으신거죠',
 19: '추천가능한 곳 뭔고',
 20: '지리는 헤어샵 있나',
 21: '실패하지않는 헤어샵 뭐있어요',
 22: '걱정없는 헤어샵 알려주시겠어요',
 23: '걱정없는 헤어샵 있다면서요',
 24: '추천해줄만한 곳 있는가봐요',
 25: '데려오고 싶은 데 뭔데에',
 26: '좋은 미용실 있는가요',
 27: '실패하지않는 미용실 있나봐예',
 28: '추천해줄만한 미용실 부탁드려요',
 29: '알려주고 싶은 곳 있는가예',
 30: '괜찮은 헤어샵 알려주실래요',
 31: '실패한적 없는 곳 뭘까요',
 32: '데려오고 싶은 미용실 가르쳐주세요',
 33: '추천할 헤어샵 알려줄수 있나요',
 34: '추천할만한 미용실 어딥니까',
 35: '오지는 데 알려줘요',
 36: '추천가능한 곳 알려주시겠습니까',
 37: '오지는 데 어데로가는교',
 38: '걱정없는 헤어샵 있으신가요',
 39: '데려오고 싶은 헤어샵 알려주시겠습니까',
 40: '예쁘게 하는 헤어샵 어데로가나',
 41: '요즘 잘나가는 곳 어디입니까',
 42: '걱정없는 미용실 물어볼려고요',
 43: '쩌는 곳 어디에있죠',
 44: '요즘 잘나가는 헤어샵 알려

In [658]:
total_adjs_df

,total_adjs,main
0,불결하지 않은,sanitation
1,빛이 나는,sanitation
2,매일 청소하는,sanitation
3,비위생적이지 않은,sanitation
4,깨끗한,sanitation
...,...,...
240,봄에 할만한,etc
241,요즘 제일 잘나가는,etc
242,파티에 하고 갈,etc
243,상견례용으로 할,etc


In [659]:
#본격적인 추천용 질문 생성
#유형 1. 형용사 + 카테고리1 + 추천 어미
#유형 2. 형용사 + 카테고리2 + 추천 어미
#유형 3. 형용사+ 추천 어미
#유형 4. 카테고리1 + 추천 어미
#유형 5. 카테고리2 + 추천 어미
#유형 6. 지역+ 추천 어미 
#(이때, 지역은 district + neighborhood + nearby_station 을 모두 합한 거에서 랜덤하게 한다)
q_type_1 = 245*3*12875
q_type_2 = 245*70*12875
q_type_3 = 245*12875
q_type_4 = 3*12875
q_type_5 = 70*12875
# q_type_6 = 
total_q = q_type_1+q_type_2+q_type_3+q_type_4+q_type_5
total_q #만들수 있는 질문 수 2.3억개 이상/ 지나치게 문장이 많이 생성되므로 랜덤하게 추출하자

234363625

In [660]:
import random
random_nums = random.sample(range(0,12875), 20)
for i in random_nums:
    print(req_list_dict[i]) #추천 질문셋 가운데 20개만 무작위로 가져옴

쩌는 미용실 있나
훌륭한 곳 있어예
훌륭한 데 어디에있죠
괜찮은 미용실 아시나요
지리는 미용실 여쭤볼려고
최고인 미용실 물어볼려고요
괜찮은 헤어샵 뭐가있죠
지리는 곳 물어볼게요
여쭤볼라고
실패한적 없는 곳 뭐예요
최고로 잘하는 곳 여쭈어볼려고요
데려오고 싶은 데 어데로가나
최고인 곳 뭐드라
솜씨좋은 헤어샵 알려주시겠어요
헤어 맛집 여쭤볼라고
하고싶은 데 알려줘요
훌륭한 헤어샵 어디인가요
걱정없는 데 없어요
같이오고 싶은 곳 알아보는데요
추천해줄만한 곳 있었나요


In [661]:
total_adjs_df.head()

,total_adjs,main
0,불결하지 않은,sanitation
1,빛이 나는,sanitation
2,매일 청소하는,sanitation
3,비위생적이지 않은,sanitation
4,깨끗한,sanitation


In [662]:
df_hair.head()

,store_name,district,neighborhood,nearby_station,hair_category1,hair_category2
0,준오헤어 노원사거리점,노원구,"상계동,상계6.7동",노원역,"머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
5947,차홍룸 홍대점,마포구,"서교동,서교동","합정역,망원역,상수역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
10731,차홍룸 용산센트럴점,용산구,"한강로2가,한강로동","신용산역,용산역,삼각지역,이촌역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
16159,차홍룸 신논현점,서초구,"서초동,서초4동","신논현역,강남역,사평역,논현역,언주역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
21098,CHOP헤어 신천점,송파구,"잠실동,잠실본동","종합운동장역,잠실새내역,삼전역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."


In [663]:
hair_type = df_hair.hair_category1[0].split(',')
hair_type

['머리', '컷', '펌']

In [664]:
hair_name = df_hair.hair_category2[0].split(',')
hair_name

['댄디컷',
 '리젠트컷',
 '스왓컷',
 '크롭컷',
 '포마드컷',
 '쉼표머리',
 '투블럭',
 '모히칸',
 '머쉬룸컷',
 '레이어드컷',
 '울프컷',
 '언밸런스컷',
 '히메컷',
 '보브컷',
 '샤기컷',
 '허쉬컷',
 '처피뱅',
 '러블리펌',
 'C컬펌',
 'S컬펌',
 '물결펌',
 '히피펌',
 '글램펌',
 '트위스트펌',
 '발롱펌',
 '베이비펌',
 '바디펌',
 '루즈펌',
 '가르마펌',
 '쉐도우펌',
 '포마드펌',
 '스핀스왈로우 펌',
 '애즈펌',
 '리젠트펌',
 '다운펌',
 '드레드락',
 '호일펌',
 '볼륨매직펌',
 '매직펌',
 '에어펌',
 '내츄럴펌',
 '바그펌',
 '보니펌',
 '레인펌',
 '프릴펌',
 '페미닌펌',
 '이지펌',
 '아이롱펌',
 '보울컷',
 '리프컷',
 '고데기펌',
 '웨이브펌',
 '복구펌',
 '젤리펌',
 '볼륨매직',
 '픽시컷',
 '보브컷',
 '언발란스컷',
 '가일컷',
 '가일펌',
 '그레이스펌',
 '태슬컷',
 '디자인펌',
 '붙임머리',
 '셋팅펌',
 '엘리자벳펌',
 '리프펌',
 '스핀스왈로펌',
 '시스루뱅',
 '투블럭펌']

In [665]:
#본격적인 추천용 질문 생성
#유형 1. 형용사 + 카테고리1 + 추천 어미
#유형 2. 형용사 + 카테고리2 + 추천 어미
#유형 3. 형용사+ 추천 어미
#유형 4. 카테고리1 + 추천 어미
#유형 5. 카테고리2 + 추천 어미

#유형 6번 일단 보류!

#유형 6. 지역+ 추천 어미 
#(이때, 지역은 district + neighborhood + nearby_station 을 모두 합한 거에서 랜덤하게 한다)

In [666]:
# 1)추천 질문 어미를 결합시켜 req_qes 란 빈 리스트에 저장
# 2) 각 질문의 형용사 / main에 해당하는 태그를 각 빈 리스트에 저장
# 3) 질문 / 메인 리스트를 req_qes_df 라는 df에 저장
req_qes = []
main_adjs= []
hair_category_1=[]
hair_category_2=[]
# 유형 1. 형용사 + 헤어 카테고리1 + 추천 어미
for adj, main_type in zip(total_adjs_df.total_adjs, total_adjs_df.main):
    adj_word = adj
    for hair_cat in df_hair.hair_category1[0].split(','):
        cat_ques = adj_word + ' ' + hair_cat
        random_nums = random.sample(range(0,12875), 60)
        for i in random_nums:
            cate_question = cat_ques + ' ' + req_list_dict[i]
            req_qes.append(cate_question) 
            main_adjs.append(main_type)
            hair_category_1.append(hair_cat)
            hair_category_2.append(np.nan)
            
# 유형 2. 형용사 + 헤어 카테고리2 + 추천 어미
for adj, main_type in zip(total_adjs_df.total_adjs, total_adjs_df.main):
    adj_word = adj
    for hair_cat in df_hair.hair_category2[0].split(','):
        cat_ques = adj_word + ' ' + hair_cat
        random_nums = random.sample(range(0,12875), 60)
        for i in random_nums:
            cate_question = cat_ques + ' ' + req_list_dict[i]
            req_qes.append(cate_question) 
            main_adjs.append(main_type)
            hair_category_1.append(np.nan)
            hair_category_2.append(hair_cat)
                       
#유형 3. 형용사+ 추천 어미
for adj, main_type in zip(total_adjs_df.total_adjs, total_adjs_df.main):
    adj_word = adj
    random_nums = random.sample(range(0,12875), 60)
    for i in random_nums:
        cate_question = adj_word + ' ' + req_list_dict[i]
        req_qes.append(cate_question) 
        main_adjs.append(main_type)
        hair_category_1.append(np.nan)
        hair_category_2.append(np.nan)
        
#유형 4. 카테고리1 + 추천 어미
for hair_cat in df_hair.hair_category1[0].split(','):
#     hair_cat = hair_cat.random.sample(hair_cat, 10)
    cat_ques = hair_cat
    random_nums = random.sample(range(0,12875), 60)
    for i in random_nums:
        cate_question = cat_ques + ' ' + req_list_dict[i]
        req_qes.append(cate_question) 
        main_adjs.append(np.nan)
        hair_category_1.append(hair_cat)
        hair_category_2.append(np.nan)
        
#유형 5. 카테고리2 + 추천 어미
for hair_cat in df_hair.hair_category2[0].split(','):
#     hair_cat = hair_cat.random.sample(hair_cat, 10)
    cat_ques = hair_cat
    random_nums = random.sample(range(0,12875), 60)
    for i in random_nums:
        cate_question = cat_ques + ' ' + req_list_dict[i]
        req_qes.append(cate_question) 
        main_adjs.append(np.nan)
        hair_category_1.append(np.nan)
        hair_category_2.append(hair_cat)

In [667]:
req_question = pd.DataFrame({
    'question' : req_qes
    ,'tag_intention' : 'recommendation'
    ,'tag_adjs' : main_adjs
    ,'tag_hair_1' : hair_category_1
    ,'tag_hair_2' : hair_category_2
})
req_question[1050:1100]

,question,tag_intention,tag_adjs,tag_hair_1,tag_hair_2
1050,정리 정돈된 펌 요즘 잘나가는 데 있으세요,recommendation,sanitation,펌,NaN
1051,정리 정돈된 펌 알려주고 싶은 데 있을까요,recommendation,sanitation,펌,NaN
1052,정리 정돈된 펌 추천할만한 데 어디냐,recommendation,sanitation,펌,NaN
1053,정리 정돈된 펌 오지는 헤어샵 뭔가요,recommendation,sanitation,펌,NaN
1054,정리 정돈된 펌 추천해줄만한 헤어샵 있는가,recommendation,sanitation,펌,NaN
1055,정리 정돈된 펌 추천해줄만한 헤어샵 알려줄래,recommendation,sanitation,펌,NaN
1056,정리 정돈된 펌 최고로 잘하는 곳 어디에요,recommendation,sanitation,펌,NaN
1057,정리 정돈된 펌 실패하지않는 곳 여쭤볼려구요,recommendation,sanitation,펌,NaN
1058,정리 정돈된 펌 훌륭한 데 부탁해,recommendation,sanitation,펌,NaN
1059,정리 정돈된 펌 추천할 미용실 뭐있어요,recommendation,sanitation,펌,NaN


In [668]:
len(req_question)

1092180

In [669]:
#추천 질문을 골고루 섞어서 학습 시킬 예정
from sklearn.utils import shuffle
req_question = shuffle(req_question)

In [670]:
req_question[350:360]

,question,tag_intention,tag_adjs,tag_hair_1,tag_hair_2
379782,특가의 셋팅펌 훌륭한 데 어디인데요,recommendation,price,NaN,셋팅펌
834269,단정해보이는 레이어드컷 추천해줄만한 곳 여쭤볼려고,recommendation,satisfaction,NaN,레이어드컷
562981,진짜 최고의 매직펌 지리는 미용실 있었나요,recommendation,satisfaction,NaN,매직펌
630593,신뢰도 높은 프릴펌 최고인 곳 뭐드라,recommendation,satisfaction,NaN,프릴펌
302222,어둡지 않은 애즈펌 걱정없는 미용실 알려주시겠습니까,recommendation,mood,NaN,애즈펌
671114,명성이 자자한 물결펌 최고인 데 알려줄수 있나요,recommendation,satisfaction,NaN,물결펌
724455,여심 저격할만한 투블럭펌 어데인지아나,recommendation,satisfaction,NaN,투블럭펌
562974,진짜 최고의 볼륨매직펌 예쁘게 하는 데 어디에있어요,recommendation,satisfaction,NaN,볼륨매직펌
502867,만족할만한 처피뱅 같이오고 싶은 데 알고계신가요,recommendation,satisfaction,NaN,처피뱅
250804,모던한 허쉬컷 추천해줄만한 미용실 여쭤보려고요,recommendation,mood,NaN,허쉬컷


## 문의 질문 셋 만들기 => 1)위치 / 2)가격 / 3)리뷰,후기

In [671]:
df_hair.head()

,store_name,district,neighborhood,nearby_station,hair_category1,hair_category2
0,준오헤어 노원사거리점,노원구,"상계동,상계6.7동",노원역,"머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
5947,차홍룸 홍대점,마포구,"서교동,서교동","합정역,망원역,상수역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
10731,차홍룸 용산센트럴점,용산구,"한강로2가,한강로동","신용산역,용산역,삼각지역,이촌역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
16159,차홍룸 신논현점,서초구,"서초동,서초4동","신논현역,강남역,사평역,논현역,언주역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
21098,CHOP헤어 신천점,송파구,"잠실동,잠실본동","종합운동장역,잠실새내역,삼전역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."


In [672]:
# df_store = df_hairshop['store_name'].unique().tolist()

In [673]:
# len(df_store)

In [674]:
# for station in df_hair.nearby_station:
#     station=str(random.choice(station))
# print(station)

In [675]:
#만들어 놓은 문의 질문 set 불러오기
with open('./que_loc_list_dict.pickle', 'rb') as f:
    que_loc_list_dict = pickle.load(f)
with open('./que_price_list_dict.pickle', 'rb') as f:
    que_price_list_dict = pickle.load(f)
with open('./que_review_list_dict.pickle', 'rb') as f:
    que_review_list_dict = pickle.load(f)

In [676]:
len(que_loc_list_dict), len(que_price_list_dict), len(que_review_list_dict)

(459, 495, 3573)

In [677]:
total_district = ','.join(df_hair.district)
print(total_district)

노원구,마포구,용산구,서초구,송파구,양천구,서대문구,용산구,서대문구,송파구,송파구,마포구,마포구,서대문구,관악구,관악구,관악구,양천구,중랑구,동작구,강남구,종로구,서초구,광진구,강남구,중구,서초구,서초구,노원구,마포구,마포구,마포구,관악구,서초구,동작구,관악구,중랑구,서초구,중구,강남구,구로구,종로구,광진구,송파구,성북구,구로구,관악구,관악구,양천구,성북구,영등포구,성동구,송파구,강서구,성동구,강서구,중구,중구,송파구,서초구,강북구,서대문구,송파구,강남구,광진구,동대문구,송파구,구로구,용산구,성북구,노원구,마포구,용산구,종로구,영등포구,중구,강남구,서초구,종로구,송파구,서대문구,강남구,강동구,강동구,강남구,중랑구,중랑구,서초구,구로구,중구,은평구,금천구,마포구,마포구,마포구,구로구,관악구,관악구,서초구,서초구,송파구,강남구,강남구,강남구,강서구,강북구,성북구,서초구,성북구,강남구,종로구,서초구,마포구,송파구,은평구,은평구,은평구,서초구,노원구,노원구,영등포구,관악구,강서구,강남구,강서구,중랑구,구로구,중구,마포구,은평구,강동구,영등포구,강동구,은평구,송파구,은평구,동작구,노원구,마포구,광진구,광진구,마포구,마포구,마포구,중구,서초구,양천구,강남구,서초구,서초구,강동구,강서구,성동구,성동구,성동구,강남구,영등포구,구로구,은평구,은평구,구로구,동대문구,마포구,용산구,중랑구,금천구,마포구,금천구,광진구,서초구,송파구,은평구,광진구,광진구,강동구,용산구,동작구,마포구,강서구,성북구,관악구,마포구,중랑구,마포구,강북구,송파구,강서구,송파구,관악구,강동구,강동구,영등포구,서초구,성북구,은평구,동작구,서대문구,성동구,은평구,은평구,마포구,강남구,강북구,동작구,동작구,광진구,강서구,서대문구,서대문구,성동구,노원구,강남구,광진구,광진구,동작구,서대문구,노원구,관악구,송파구,중랑구,관악구,관악구,영등포구,강남구,동대문구,강남구,강서구,성북구,서초구,송파구,서초구,송파구,서대문구,강서구,노원구,서대문구,영등포구,성북구,서대문구,마포구,송파구,노원구,서대문구,마포구,광진구,광진구,

In [678]:
total_district.split(',')

['노원구',
 '마포구',
 '용산구',
 '서초구',
 '송파구',
 '양천구',
 '서대문구',
 '용산구',
 '서대문구',
 '송파구',
 '송파구',
 '마포구',
 '마포구',
 '서대문구',
 '관악구',
 '관악구',
 '관악구',
 '양천구',
 '중랑구',
 '동작구',
 '강남구',
 '종로구',
 '서초구',
 '광진구',
 '강남구',
 '중구',
 '서초구',
 '서초구',
 '노원구',
 '마포구',
 '마포구',
 '마포구',
 '관악구',
 '서초구',
 '동작구',
 '관악구',
 '중랑구',
 '서초구',
 '중구',
 '강남구',
 '구로구',
 '종로구',
 '광진구',
 '송파구',
 '성북구',
 '구로구',
 '관악구',
 '관악구',
 '양천구',
 '성북구',
 '영등포구',
 '성동구',
 '송파구',
 '강서구',
 '성동구',
 '강서구',
 '중구',
 '중구',
 '송파구',
 '서초구',
 '강북구',
 '서대문구',
 '송파구',
 '강남구',
 '광진구',
 '동대문구',
 '송파구',
 '구로구',
 '용산구',
 '성북구',
 '노원구',
 '마포구',
 '용산구',
 '종로구',
 '영등포구',
 '중구',
 '강남구',
 '서초구',
 '종로구',
 '송파구',
 '서대문구',
 '강남구',
 '강동구',
 '강동구',
 '강남구',
 '중랑구',
 '중랑구',
 '서초구',
 '구로구',
 '중구',
 '은평구',
 '금천구',
 '마포구',
 '마포구',
 '마포구',
 '구로구',
 '관악구',
 '관악구',
 '서초구',
 '서초구',
 '송파구',
 '강남구',
 '강남구',
 '강남구',
 '강서구',
 '강북구',
 '성북구',
 '서초구',
 '성북구',
 '강남구',
 '종로구',
 '서초구',
 '마포구',
 '송파구',
 '은평구',
 '은평구',
 '은평구',
 '서초구',
 '노원구',
 '노원구',
 '영등포구',
 '관악구',
 '강서구',
 '강남구',
 '강서구

In [679]:
total_district = total_district.split(',')
total_district

['노원구',
 '마포구',
 '용산구',
 '서초구',
 '송파구',
 '양천구',
 '서대문구',
 '용산구',
 '서대문구',
 '송파구',
 '송파구',
 '마포구',
 '마포구',
 '서대문구',
 '관악구',
 '관악구',
 '관악구',
 '양천구',
 '중랑구',
 '동작구',
 '강남구',
 '종로구',
 '서초구',
 '광진구',
 '강남구',
 '중구',
 '서초구',
 '서초구',
 '노원구',
 '마포구',
 '마포구',
 '마포구',
 '관악구',
 '서초구',
 '동작구',
 '관악구',
 '중랑구',
 '서초구',
 '중구',
 '강남구',
 '구로구',
 '종로구',
 '광진구',
 '송파구',
 '성북구',
 '구로구',
 '관악구',
 '관악구',
 '양천구',
 '성북구',
 '영등포구',
 '성동구',
 '송파구',
 '강서구',
 '성동구',
 '강서구',
 '중구',
 '중구',
 '송파구',
 '서초구',
 '강북구',
 '서대문구',
 '송파구',
 '강남구',
 '광진구',
 '동대문구',
 '송파구',
 '구로구',
 '용산구',
 '성북구',
 '노원구',
 '마포구',
 '용산구',
 '종로구',
 '영등포구',
 '중구',
 '강남구',
 '서초구',
 '종로구',
 '송파구',
 '서대문구',
 '강남구',
 '강동구',
 '강동구',
 '강남구',
 '중랑구',
 '중랑구',
 '서초구',
 '구로구',
 '중구',
 '은평구',
 '금천구',
 '마포구',
 '마포구',
 '마포구',
 '구로구',
 '관악구',
 '관악구',
 '서초구',
 '서초구',
 '송파구',
 '강남구',
 '강남구',
 '강남구',
 '강서구',
 '강북구',
 '성북구',
 '서초구',
 '성북구',
 '강남구',
 '종로구',
 '서초구',
 '마포구',
 '송파구',
 '은평구',
 '은평구',
 '은평구',
 '서초구',
 '노원구',
 '노원구',
 '영등포구',
 '관악구',
 '강서구',
 '강남구',
 '강서구

In [680]:
total_neighborhood = ','.join(df_hair.neighborhood)
print(total_neighborhood)

상계동,상계6.7동,서교동,서교동,한강로2가,한강로동,서초동,서초4동,잠실동,잠실본동,목동,목1동,창천동,신촌동,한남동,한남동,창천동,신촌동,잠실동,잠실본동,잠실동,잠실본동,서교동,서교동,서교동,서교동,창천동,신촌동,신림동,서원동,봉천동,청룡동,봉천동,청룡동,목동,목1동,상봉동,상봉2동,노량진동,노량진1동,대치동,대치2동,관철동,종로1.2.3.4가동,서초동,서초4동,자양동,자양4동,대치동,대치4동,충무로1가,명동,방배동,방배1동,방배동,방배1동,상계동,상계2동,서교동,서교동,동교동,서교동,동교동,서교동,신림동,신림동,서초동,서초4동,사당동,사당2동,봉천동,행운동,신내동,신내1동,방배동,방배4동,명동2가,명동,논현동,논현1동,신도림동,신도림동,혜화동,혜화동,자양동,자양4동,문정동,문정2동,동선동1가,동선동,신도림동,신도림동,봉천동,낙성대동,봉천동,낙성대동,목동,목1동,안암동5가,안암동,영등포동,영등포본동,행당동,행당1동,삼전동,삼전동,등촌동,등촌1동,행당동,행당1동,마곡동,가양1동,신당동,약수동,신당동,약수동,석촌동,석촌동,서초동,서초3동,미아동,송천동,연희동,연희동,석촌동,석촌동,삼성동,삼성1동,화양동,화양동,회기동,회기동,석촌동,석촌동,구로동,구로3동,한강로2가,한강로동,동선동1가,동선동,상계동,상계2동,동교동,서교동,보광동,보광동,관철동,종로1.2.3.4가동,영등포동3가,영등포동,을지로6가,광희동,역삼동,역삼1동,서초동,서초2동,동숭동,이화동,가락동,가락1동,홍제동,홍제1동,역삼동,역삼1동,천호동,천호3동,천호동,천호3동,대치동,대치1동,상봉동,상봉2동,면목동,면목7동,서초동,서초4동,구로동,구로5동,충무로1가,명동,갈현동,갈현1동,가산동,가산동,합정동,합정동,동교동,서교동,동교동,서교동,오류동,오류1동,봉천동,청룡동,봉천동,청룡동,서초동,서초4동,반포동,반포4동,가락동,가락1동,역삼동,역삼1동,역삼동,역삼1동,대치동,대치1동,마곡동,방화1동,미아동,송중동,동소문동5가,삼선동,서초동,서초4동,길음동,길음2동,대치동,대치2동,관철동,종로1.2.3.4가동,반포

In [681]:
total_neighborhood.split(',')

['상계동',
 '상계6.7동',
 '서교동',
 '서교동',
 '한강로2가',
 '한강로동',
 '서초동',
 '서초4동',
 '잠실동',
 '잠실본동',
 '목동',
 '목1동',
 '창천동',
 '신촌동',
 '한남동',
 '한남동',
 '창천동',
 '신촌동',
 '잠실동',
 '잠실본동',
 '잠실동',
 '잠실본동',
 '서교동',
 '서교동',
 '서교동',
 '서교동',
 '창천동',
 '신촌동',
 '신림동',
 '서원동',
 '봉천동',
 '청룡동',
 '봉천동',
 '청룡동',
 '목동',
 '목1동',
 '상봉동',
 '상봉2동',
 '노량진동',
 '노량진1동',
 '대치동',
 '대치2동',
 '관철동',
 '종로1.2.3.4가동',
 '서초동',
 '서초4동',
 '자양동',
 '자양4동',
 '대치동',
 '대치4동',
 '충무로1가',
 '명동',
 '방배동',
 '방배1동',
 '방배동',
 '방배1동',
 '상계동',
 '상계2동',
 '서교동',
 '서교동',
 '동교동',
 '서교동',
 '동교동',
 '서교동',
 '신림동',
 '신림동',
 '서초동',
 '서초4동',
 '사당동',
 '사당2동',
 '봉천동',
 '행운동',
 '신내동',
 '신내1동',
 '방배동',
 '방배4동',
 '명동2가',
 '명동',
 '논현동',
 '논현1동',
 '신도림동',
 '신도림동',
 '혜화동',
 '혜화동',
 '자양동',
 '자양4동',
 '문정동',
 '문정2동',
 '동선동1가',
 '동선동',
 '신도림동',
 '신도림동',
 '봉천동',
 '낙성대동',
 '봉천동',
 '낙성대동',
 '목동',
 '목1동',
 '안암동5가',
 '안암동',
 '영등포동',
 '영등포본동',
 '행당동',
 '행당1동',
 '삼전동',
 '삼전동',
 '등촌동',
 '등촌1동',
 '행당동',
 '행당1동',
 '마곡동',
 '가양1동',
 '신당동',
 '약수동',
 '신당동',
 '약수동',
 '석촌동',
 '석촌동',
 '서초동

In [682]:
total_neighborhood = total_neighborhood.split(',')
total_neighborhood

['상계동',
 '상계6.7동',
 '서교동',
 '서교동',
 '한강로2가',
 '한강로동',
 '서초동',
 '서초4동',
 '잠실동',
 '잠실본동',
 '목동',
 '목1동',
 '창천동',
 '신촌동',
 '한남동',
 '한남동',
 '창천동',
 '신촌동',
 '잠실동',
 '잠실본동',
 '잠실동',
 '잠실본동',
 '서교동',
 '서교동',
 '서교동',
 '서교동',
 '창천동',
 '신촌동',
 '신림동',
 '서원동',
 '봉천동',
 '청룡동',
 '봉천동',
 '청룡동',
 '목동',
 '목1동',
 '상봉동',
 '상봉2동',
 '노량진동',
 '노량진1동',
 '대치동',
 '대치2동',
 '관철동',
 '종로1.2.3.4가동',
 '서초동',
 '서초4동',
 '자양동',
 '자양4동',
 '대치동',
 '대치4동',
 '충무로1가',
 '명동',
 '방배동',
 '방배1동',
 '방배동',
 '방배1동',
 '상계동',
 '상계2동',
 '서교동',
 '서교동',
 '동교동',
 '서교동',
 '동교동',
 '서교동',
 '신림동',
 '신림동',
 '서초동',
 '서초4동',
 '사당동',
 '사당2동',
 '봉천동',
 '행운동',
 '신내동',
 '신내1동',
 '방배동',
 '방배4동',
 '명동2가',
 '명동',
 '논현동',
 '논현1동',
 '신도림동',
 '신도림동',
 '혜화동',
 '혜화동',
 '자양동',
 '자양4동',
 '문정동',
 '문정2동',
 '동선동1가',
 '동선동',
 '신도림동',
 '신도림동',
 '봉천동',
 '낙성대동',
 '봉천동',
 '낙성대동',
 '목동',
 '목1동',
 '안암동5가',
 '안암동',
 '영등포동',
 '영등포본동',
 '행당동',
 '행당1동',
 '삼전동',
 '삼전동',
 '등촌동',
 '등촌1동',
 '행당동',
 '행당1동',
 '마곡동',
 '가양1동',
 '신당동',
 '약수동',
 '신당동',
 '약수동',
 '석촌동',
 '석촌동',
 '서초동

In [683]:
test=df_hair.nearby_station.dropna().values
test

array(['노원역', '합정역,망원역,상수역', '신용산역,용산역,삼각지역,이촌역', ..., '신사역,논현역,잠원역',
       '송파역,석촌역,가락시장역', '신촌역,서강대역,이대역,홍대입구역'], dtype=object)

In [684]:
len(test)

2651

In [685]:
test_1 = ','.join(test) #지하철 nan 108개 있음
test_1

'노원역,합정역,망원역,상수역,신용산역,용산역,삼각지역,이촌역,신논현역,강남역,사평역,논현역,언주역,종합운동장역,잠실새내역,삼전역,오목교역,양평역,목동역,신촌역,서강대역,이대역,홍대입구역,이태원역,한강진역,신촌역,서강대역,이대역,홍대입구역,잠실새내역,종합운동장역,삼전역,잠실새내역,종합운동장역,삼전역,상수역,합정역,홍대입구역,합정역,상수역,홍대입구역,망원역,신촌역,서강대역,이대역,홍대입구역,신림역,서울대입구역,봉천역,서울대입구역,봉천역,오목교역,목동역,중랑역,상봉역,노량진역,장승배기역,대치역,한티역,삼성역,학여울역,종각역,종로3가역,을지로3가역,을지로입구역,안국역,을지로4가역,명동역,광화문역,신논현역,강남역,사평역,언주역,역삼역,건대입구역,한티역,도곡역,선릉역,대치역,명동역,을지로입구역,회현역,시청역,을지로3가역,충무로역,방배역,내방역,방배역,내방역,노원역,홍대입구역,망원역,합정역,홍대입구역,상수역,홍대입구역,상수역,신림역,신논현역,강남역,사평역,논현역,총신대입구역,이수역,사당역,남성역,서울대입구역,망우역,총신대입구역,이수역,내방역,사당역,명동역,을지로입구역,회현역,시청역,을지로3가역,종각역,충무로역,신논현역,논현역,사평역,언주역,신도림역,도림천역,문래역,혜화역,한성대입구역,건대입구역,뚝섬유원지역,장지역,문정역,성신여대입구역,보문역,신도림역,도림천역,구로역,서울대입구역,낙성대역,서울대입구역,낙성대역,오목교역,목동역,안암역,고려대역,보문역,제기동역,영등포역,신길역,영등포시장역,왕십리역,행당역,상왕십리역,한양대역,마장역,삼전역,잠실새내역,석촌고분역,증미역,가양역,왕십리역,마장역,한양대역,상왕십리역,행당역,마곡나루역,신방화역,마곡역,약수역,버티고개역,동대입구역,금호역,청구역,신금호역,약수역,버티고개역,동대입구역,금호역,청구역,신금호역,송파역,석촌역,남부터미널역,교대역,서초역,미아사거리역,석촌고분역,석촌역,잠실역,삼성역,봉은사역,삼성중앙역,선릉역,건대입구역,어린이대공원역,회기역,석촌역,송파나루역,잠실역,석촌고분역,송파역,구로디지털단지역,신용산역,용산역,삼각지역,이촌역,

In [686]:
a = '합정역, 노원역, 강남역'

len(a)

13

In [687]:
len(test_1) #' ' 안에 전체 문자열의 길이

34453

In [688]:
test_2=test_1.split(',')

In [689]:
test_2

['노원역',
 '합정역',
 '망원역',
 '상수역',
 '신용산역',
 '용산역',
 '삼각지역',
 '이촌역',
 '신논현역',
 '강남역',
 '사평역',
 '논현역',
 '언주역',
 '종합운동장역',
 '잠실새내역',
 '삼전역',
 '오목교역',
 '양평역',
 '목동역',
 '신촌역',
 '서강대역',
 '이대역',
 '홍대입구역',
 '이태원역',
 '한강진역',
 '신촌역',
 '서강대역',
 '이대역',
 '홍대입구역',
 '잠실새내역',
 '종합운동장역',
 '삼전역',
 '잠실새내역',
 '종합운동장역',
 '삼전역',
 '상수역',
 '합정역',
 '홍대입구역',
 '합정역',
 '상수역',
 '홍대입구역',
 '망원역',
 '신촌역',
 '서강대역',
 '이대역',
 '홍대입구역',
 '신림역',
 '서울대입구역',
 '봉천역',
 '서울대입구역',
 '봉천역',
 '오목교역',
 '목동역',
 '중랑역',
 '상봉역',
 '노량진역',
 '장승배기역',
 '대치역',
 '한티역',
 '삼성역',
 '학여울역',
 '종각역',
 '종로3가역',
 '을지로3가역',
 '을지로입구역',
 '안국역',
 '을지로4가역',
 '명동역',
 '광화문역',
 '신논현역',
 '강남역',
 '사평역',
 '언주역',
 '역삼역',
 '건대입구역',
 '한티역',
 '도곡역',
 '선릉역',
 '대치역',
 '명동역',
 '을지로입구역',
 '회현역',
 '시청역',
 '을지로3가역',
 '충무로역',
 '방배역',
 '내방역',
 '방배역',
 '내방역',
 '노원역',
 '홍대입구역',
 '망원역',
 '합정역',
 '홍대입구역',
 '상수역',
 '홍대입구역',
 '상수역',
 '신림역',
 '신논현역',
 '강남역',
 '사평역',
 '논현역',
 '총신대입구역',
 '이수역',
 '사당역',
 '남성역',
 '서울대입구역',
 '망우역',
 '총신대입구역',
 '이수역',
 '내방역',
 '사당역',
 '명동역',
 '을지로입구역',
 '

In [690]:
len(test_2)

7103

In [691]:
set(test_2)

{'4.19민주묘지역',
 '가락시장역',
 '가산디지털단지역',
 '가양역',
 '가오리역',
 '가좌역',
 '강남구청역',
 '강남역',
 '강동구청역',
 '강동역',
 '강변역',
 '개롱역',
 '개봉역',
 '개포동역',
 '개화산역',
 '거여역',
 '건대입구역',
 '경복궁역',
 '경찰병원역',
 '고덕역',
 '고려대역',
 '고속터미널역',
 '공덕역',
 '공릉역',
 '공항시장역',
 '광나루역',
 '광명사거리역',
 '광운대역',
 '광화문역',
 '광흥창역',
 '교대역',
 '구로디지털단지역',
 '구로역',
 '구룡역',
 '구반포역',
 '구산역',
 '구의역',
 '구일역',
 '구파발역',
 '국회의사당역',
 '군자역',
 '굽은다리역',
 '금천구청역',
 '금호역',
 '길동역',
 '길음역',
 '김포공항역',
 '까치산역',
 '낙성대역',
 '남구로역',
 '남부터미널역',
 '남성역',
 '남영역',
 '남태령역',
 '내방역',
 '노들역',
 '노량진역',
 '노원역',
 '녹번역',
 '녹사평역',
 '녹천역',
 '논현역',
 '답십리역',
 '당산역',
 '대림역',
 '대모산입구역',
 '대방역',
 '대청역',
 '대치역',
 '대흥역',
 '도곡역',
 '도림천역',
 '도봉산역',
 '도봉역',
 '독립문역',
 '독바위역',
 '독산역',
 '돌곶이역',
 '동대문역',
 '동대문역사문화공원역',
 '동대입구역',
 '동묘앞역',
 '동작역',
 '둔촌동역',
 '둔촌오륜역',
 '등촌역',
 '디지털미디어시티역',
 '뚝섬역',
 '뚝섬유원지역',
 '마곡나루역',
 '마곡역',
 '마들역',
 '마장역',
 '마천역',
 '마포구청역',
 '마포역',
 '망우역',
 '망원역',
 '매봉역',
 '먹골역',
 '면목역',
 '명동역',
 '명일역',
 '목동역',
 '몽촌토성역',
 '무악재역',
 '문래역',
 '문정역',
 '미아사거리역',
 '미아역',
 '반포역',
 '발산역',

In [692]:
len(set(test_2))

295

In [693]:
test_3 = set(test_2)

In [694]:
test_3

{'4.19민주묘지역',
 '가락시장역',
 '가산디지털단지역',
 '가양역',
 '가오리역',
 '가좌역',
 '강남구청역',
 '강남역',
 '강동구청역',
 '강동역',
 '강변역',
 '개롱역',
 '개봉역',
 '개포동역',
 '개화산역',
 '거여역',
 '건대입구역',
 '경복궁역',
 '경찰병원역',
 '고덕역',
 '고려대역',
 '고속터미널역',
 '공덕역',
 '공릉역',
 '공항시장역',
 '광나루역',
 '광명사거리역',
 '광운대역',
 '광화문역',
 '광흥창역',
 '교대역',
 '구로디지털단지역',
 '구로역',
 '구룡역',
 '구반포역',
 '구산역',
 '구의역',
 '구일역',
 '구파발역',
 '국회의사당역',
 '군자역',
 '굽은다리역',
 '금천구청역',
 '금호역',
 '길동역',
 '길음역',
 '김포공항역',
 '까치산역',
 '낙성대역',
 '남구로역',
 '남부터미널역',
 '남성역',
 '남영역',
 '남태령역',
 '내방역',
 '노들역',
 '노량진역',
 '노원역',
 '녹번역',
 '녹사평역',
 '녹천역',
 '논현역',
 '답십리역',
 '당산역',
 '대림역',
 '대모산입구역',
 '대방역',
 '대청역',
 '대치역',
 '대흥역',
 '도곡역',
 '도림천역',
 '도봉산역',
 '도봉역',
 '독립문역',
 '독바위역',
 '독산역',
 '돌곶이역',
 '동대문역',
 '동대문역사문화공원역',
 '동대입구역',
 '동묘앞역',
 '동작역',
 '둔촌동역',
 '둔촌오륜역',
 '등촌역',
 '디지털미디어시티역',
 '뚝섬역',
 '뚝섬유원지역',
 '마곡나루역',
 '마곡역',
 '마들역',
 '마장역',
 '마천역',
 '마포구청역',
 '마포역',
 '망우역',
 '망원역',
 '매봉역',
 '먹골역',
 '면목역',
 '명동역',
 '명일역',
 '목동역',
 '몽촌토성역',
 '무악재역',
 '문래역',
 '문정역',
 '미아사거리역',
 '미아역',
 '반포역',
 '발산역',

In [695]:
total_station=list(test_3)
total_station

['도곡역',
 '구로디지털단지역',
 '숙대입구역',
 '서울역',
 '남구로역',
 '숭실대입구역',
 '대모산입구역',
 '상월곡역',
 '장승배기역',
 '불광역',
 '개롱역',
 '증미역',
 '금호역',
 '봉은사역',
 '녹천역',
 '남영역',
 '합정역',
 '선유도역',
 '길음역',
 '양재시민의숲역',
 '양재역',
 '여의도역',
 '외대앞역',
 '서초역',
 '가좌역',
 '압구정역',
 '고덕역',
 '개화산역',
 '마천역',
 '상수역',
 '안암역',
 '서울숲역',
 '오류동역',
 '학여울역',
 '독산역',
 '길동역',
 '신논현역',
 '성신여대입구역',
 '옥수역',
 '복정역',
 '사가정역',
 '먹골역',
 '망원역',
 '목동역',
 '장지역',
 '회기역',
 '신당역',
 '명동역',
 '송파나루역',
 '노량진역',
 '종로5가역',
 '상봉역',
 '청구역',
 '영등포역',
 '화곡역',
 '화계역',
 '아현역',
 '구룡역',
 '버티고개역',
 '용두역',
 '송파역',
 '아차산역',
 '정릉역',
 '상도역',
 '광운대역',
 '고속터미널역',
 '돌곶이역',
 '몽촌토성역',
 '신대방역',
 '당산역',
 '왕십리역',
 '대치역',
 '제기동역',
 '응암역',
 '행당역',
 '오목교역',
 '새절역',
 '공릉역',
 '구반포역',
 '보라매역',
 '광나루역',
 '시청역',
 '내방역',
 '등촌역',
 '중곡역',
 '송정역',
 '응봉역',
 '홍대입구역',
 '연신내역',
 '장한평역',
 '봉천역',
 '거여역',
 '화랑대역',
 '녹번역',
 '청담역',
 '김포공항역',
 '을지로입구역',
 '신답역',
 '회현역',
 '약수역',
 '마포역',
 '월곡역',
 '사당역',
 '노들역',
 '망우역',
 '염창역',
 '양평역',
 '독바위역',
 '우장산역',
 '강변역',
 '봉화산역6호선',
 '방학역',
 '가오리역',
 '개봉역',
 '총신대입구역',
 

In [696]:
#질문 관련하여 장소어미 459, 가격관련 495, 리뷰관련 3573개 존재
#장소 문의용 질문 생성
#유형 1. 업소명 + 장소 문의 어미
#유형 2. 구(district) + 장소 문의 어미
#유형 3. 동(neighborhood)+ + 장소 문의 어미
#유형 4. 근처 지하철 역(nearby_station)+ 장소 문의 어미

#유형 5은 일단 보류

#유형 5. 지역+ 장소 문의 어미 
#(이때, 지역은 district + neighborhood + nearby_station 을 모두 합한 거에서 랜덤하게 한다)

#유형 1. 업소명 + 장소 문의 어미
q_about_loc = []

for name in df_store:
    storename = name
    random_nums = random.sample(range(0,459), 50) #50개 랜덤 장소관련 문의 질문 
    for i in random_nums:
        long_question = storename + ' ' + que_loc_list_dict[i]
        q_about_loc.append(long_question)

#유형 2. 구(district) + 장소 문의 어미
for district in df_hair.district:
    random_nums = random.sample(range(0,459), 40) #50개 랜덤 장소관련 문의 질문 
    for i in random_nums:
        long_question_1 = district + '에 있는 헤어샵'+ ' ' + que_loc_list_dict[i]
        long_question_2 = district + '에 있는 미용실'+ ' ' + que_loc_list_dict[i]
        long_question_3 = district + '에 있는 미장원'+ ' ' + que_loc_list_dict[i]
        q_about_loc.append(long_question_1)
        q_about_loc.append(long_question_2)
        q_about_loc.append(long_question_3)
        
#유형 3. 동(neighborhood)+ + 장소 문의 어미
for neighborhood in total_neighborhood:
#     neighborhood=random.choice(neighborhood)
    random_nums = random.sample(range(0,459), 40) #50개 랜덤 장소관련 문의 질문 
    for i in random_nums:
        long_question_1 = neighborhood + '에 있는 헤어샵'+ ' ' + que_loc_list_dict[i]
        long_question_2 = neighborhood + '에 있는 미용실'+ ' ' + que_loc_list_dict[i]
        long_question_3 = neighborhood + '에 있는 미장원'+ ' ' + que_loc_list_dict[i]
        q_about_loc.append(long_question_1)
        q_about_loc.append(long_question_2)
        q_about_loc.append(long_question_3)
        
#유형 4. 근처 지하철 역(nearby_station)+ 장소 문의 어미
for station in total_station:
#     station=random.choice(station) #모든 역이 다 나옴. 랜덤하게 추출 하려고 많은 방법 사용했으나 해결 x
    random_nums = random.sample(range(0,459), 40) #50개 랜덤 장소관련 문의 질문 
    for i in random_nums:
        long_question_1 = str(station) + ' ' + '근처 헤어샵'+ ' ' + que_loc_list_dict[i]
        long_question_2 = str(station) + ' ' + '근처 미용실'+ ' ' + que_loc_list_dict[i]
        long_question_3 = str(station) + ' ' + '근처 미장원'+ ' ' + que_loc_list_dict[i]
        q_about_loc.append(long_question_1)
        q_about_loc.append(long_question_2)
        q_about_loc.append(long_question_3)

In [697]:
q_about_loc[234500:234510]

['강서구에 있는 미장원 장소 어디있어요',
 '강서구에 있는 헤어샵 자세한 위치 어디지',
 '강서구에 있는 미용실 자세한 위치 어디지',
 '강서구에 있는 미장원 자세한 위치 어디지',
 '강서구에 있는 헤어샵 상세주소 어딘지아나',
 '강서구에 있는 미용실 상세주소 어딘지아나',
 '강서구에 있는 미장원 상세주소 어딘지아나',
 '강서구에 있는 헤어샵 자세한 위치 어디인지 말해줘요',
 '강서구에 있는 미용실 자세한 위치 어디인지 말해줘요',
 '강서구에 있는 미장원 자세한 위치 어디인지 말해줘요']

In [698]:
len(q_about_loc) #위치 관련 질문셋 어미 추가로 확보하여 100만개까지 만들것

1163940

In [699]:
random.shuffle(q_about_loc) #질문 리스트를 다시 한번 무작위로 섞는다

In [700]:
q_about_loc[:10]

['마포구에 있는 미장원 어디 위치해있죠',
 '강북구에 있는 헤어샵 알고 있냐',
 '바닐랄라헤어 2호점 장소 어디에있죠',
 '목동에 있는 헤어샵 찾아줘요',
 '갈현1동에 있는 헤어샵 위치 어뎁니까',
 '강남구에 있는 헤어샵 자세한 장소 찾아줘',
 '종각역 근처 미장원 장소 어디',
 '북가좌2동에 있는 미장원 주소 알고 있어',
 '목동에 있는 헤어샵 자세한 장소 어디있어요',
 '갈현1동에 있는 헤어샵 위치 어디죠']

In [701]:
#dataframe화 작업
df_ques_loc = pd.DataFrame({
    'question' : q_about_loc
    ,'tag_intention' : 'location'
    ,'tag_hair_1' : np.nan
    ,'tag_hair_2' : np.nan
})
df_ques_loc.head()

,question,tag_intention,tag_hair_1,tag_hair_2
0,마포구에 있는 미장원 어디 위치해있죠,location,NaN,NaN
1,강북구에 있는 헤어샵 알고 있냐,location,NaN,NaN
2,바닐랄라헤어 2호점 장소 어디에있죠,location,NaN,NaN
3,목동에 있는 헤어샵 찾아줘요,location,NaN,NaN
4,갈현1동에 있는 헤어샵 위치 어뎁니까,location,NaN,NaN


## 가격 문의 질문 셋 만들기

In [702]:
hair_type = df_hair.hair_category1[0].split(',')
print(hair_type)
hair_name = df_hair.hair_category2[0].split(',')
print(hair_name)

['머리', '컷', '펌']
['댄디컷', '리젠트컷', '스왓컷', '크롭컷', '포마드컷', '쉼표머리', '투블럭', '모히칸', '머쉬룸컷', '레이어드컷', '울프컷', '언밸런스컷', '히메컷', '보브컷', '샤기컷', '허쉬컷', '처피뱅', '러블리펌', 'C컬펌', 'S컬펌', '물결펌', '히피펌', '글램펌', '트위스트펌', '발롱펌', '베이비펌', '바디펌', '루즈펌', '가르마펌', '쉐도우펌', '포마드펌', '스핀스왈로우 펌', '애즈펌', '리젠트펌', '다운펌', '드레드락', '호일펌', '볼륨매직펌', '매직펌', '에어펌', '내츄럴펌', '바그펌', '보니펌', '레인펌', '프릴펌', '페미닌펌', '이지펌', '아이롱펌', '보울컷', '리프컷', '고데기펌', '웨이브펌', '복구펌', '젤리펌', '볼륨매직', '픽시컷', '보브컷', '언발란스컷', '가일컷', '가일펌', '그레이스펌', '태슬컷', '디자인펌', '붙임머리', '셋팅펌', '엘리자벳펌', '리프펌', '스핀스왈로펌', '시스루뱅', '투블럭펌']


In [703]:
df_hair

,store_name,district,neighborhood,nearby_station,hair_category1,hair_category2
0,준오헤어 노원사거리점,노원구,"상계동,상계6.7동",노원역,"머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
5947,차홍룸 홍대점,마포구,"서교동,서교동","합정역,망원역,상수역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
10731,차홍룸 용산센트럴점,용산구,"한강로2가,한강로동","신용산역,용산역,삼각지역,이촌역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
16159,차홍룸 신논현점,서초구,"서초동,서초4동","신논현역,강남역,사평역,논현역,언주역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
21098,CHOP헤어 신천점,송파구,"잠실동,잠실본동","종합운동장역,잠실새내역,삼전역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
...,...,...,...,...,...,...
815495,페이지헤어,도봉구,"도봉동,도봉2동",방학역,"머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
815496,SM프로헤어 한국외대점,동대문구,"이문동,이문1동","외대앞역,회기역,신이문역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
816376,론헤어 그루밍점,강남구,"신사동,신사동","신사역,논현역,잠원역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."
816837,살롱에이투지 헬리오시티점,송파구,"가락동,가락1동","송파역,석촌역,가락시장역","머리,컷,펌","댄디컷,리젠트컷,스왓컷,크롭컷,포마드컷,쉼표머리,투블럭,모히칸,머쉬룸컷,레이어드컷,..."


In [704]:
print('종결어미 수 :', len(que_price_list_dict))

종결어미 수 : 495


In [705]:
# 유형 1. 형용사 + 헤어 카테고리1 + 추천 어미
req_qes = []
main_adjs= []
hair_category_1=[]
hair_category_2=[]

for adj, main_type in zip(total_adjs_df.total_adjs, total_adjs_df.main):
    adj_word = adj
    for hair_cat in df_hair.hair_category1[0].split(','):
        cat_ques = adj_word + ' ' + hair_cat
        random_nums = random.sample(range(0,12875), 60)
        for i in random_nums:
            cate_question = cat_ques + ' ' + req_list_dict[i]
            req_qes.append(cate_question) 
            main_adjs.append(main_type)
            hair_category_1.append(hair_cat)
            hair_category_2.append(np.nan)

In [706]:
#가격 문의용 질문 생성

# 유형 1. 상호명 + 헤어 카테고리1 + 가격문의 어미
# 유형 2. 상호명 + 헤어 카테고리2 + 가격문의 어미
#
q_about_pri = []
hair_category_1=[]
hair_category_2=[]

# 유형 1. 상호명 + 헤어 카테고리1 + 가격문의 어미
for name in df_store:
    for hair_type in df_hair.hair_category1[0].split(','): 
        random_nums = random.sample(range(0,495), 6) # 6개 랜덤 가격 문의 질문 
        for i in random_nums:
            long_question = name + ' '+ hair_type + ' ' + que_price_list_dict[i]           
            hair_category_1.append(hair_type)
            hair_category_2.append(np.nan)
            q_about_pri.append(long_question)
            
# 유형 2. 상호명 + 헤어 카테고리2 + 가격문의 어미
for name in df_store:
    for hair_type in df_hair.hair_category2[0].split(','): 
        random_nums = random.sample(range(0,495), 6) # 6개 랜덤 가격 문의 질문 
        for i in random_nums:
            long_question = name + ' '+ hair_type + ' ' + que_price_list_dict[i]           
            hair_category_1.append(np.nan)
            hair_category_2.append(hair_type)
            q_about_pri.append(long_question)

In [707]:
zip(q_about_pri, hair_target)

In [708]:
price_complete_question = list(zip(q_about_pri, hair_category_1, hair_category_2))
price_complete_question[150:160]

[('제오헤어 신촌점 컷 하는 비용 어떠니?', '컷', nan),
 ('제오헤어 신촌점 컷 요금 물어볼려고요', '컷', nan),
 ('제오헤어 신촌점 컷 값 알수있나요', '컷', nan),
 ('제오헤어 신촌점 컷 궁금해요', '컷', nan),
 ('제오헤어 신촌점 컷 하는 비용 여쭤보려구요', '컷', nan),
 ('제오헤어 신촌점 컷 비용 궁금해서요', '컷', nan),
 ('제오헤어 신촌점 펌 액수 어떻습니까?', '펌', nan),
 ('제오헤어 신촌점 펌 값 어떻지?', '펌', nan),
 ('제오헤어 신촌점 펌 요금 어떤지요?', '펌', nan),
 ('제오헤어 신촌점 펌 가격 찾아줘', '펌', nan)]

In [709]:
len(q_about_pri)

1185228

In [710]:
q_about_pri[99500:99510]

['영등포미용실엘라인 엘리자벳펌 비용 여쭤보려고요',
 '영등포미용실엘라인 엘리자벳펌 하는 요금 찾아줄래',
 '영등포미용실엘라인 엘리자벳펌 액수 알아보려고요',
 '영등포미용실엘라인 엘리자벳펌 아시나요',
 '영등포미용실엘라인 리프펌 하는 가격 물어볼라는데요',
 '영등포미용실엘라인 리프펌 액수 어떻습니까?',
 '영등포미용실엘라인 리프펌 가격 물어봐야겠죠',
 '영등포미용실엘라인 리프펌 하는 가격 알려주세요',
 '영등포미용실엘라인 리프펌 부탁드려요',
 '영등포미용실엘라인 리프펌 비용 여쭤보려구요']

In [711]:
random.shuffle(price_complete_question)
price_complete_question

[('쇼군헤어포맨 독산본점 보브컷 비용 알려줘여', nan, '보브컷'),
 ('어반그라운드헤어 신대방삼거리점 가일컷 하는 가격 여쭤보려고요', nan, '가일컷'),
 ('라브리지헤어살롱 중계역점 펌 액수 여쭤보려구요', '펌', nan),
 ('지노헤어 숙대1호점 러블리펌 비용 여쭈어볼려고요', nan, '러블리펌'),
 ('더부띠끄 왕십리점 볼륨매직펌 알려주시겠어요', nan, '볼륨매직펌'),
 ('이정연헤어살롱 샤기컷 하는 비용 어떨까?', nan, '샤기컷'),
 ('플랜에이치 우장산9호점 그레이스펌 하는 요금 어떠니?', nan, '그레이스펌'),
 ('주빈헤어 언발란스컷 액수 어때요?', nan, '언발란스컷'),
 ('아이디헤어 은평뉴타운점 태슬컷 가격 궁금해요', nan, '태슬컷'),
 ('살롱드 바이 깁슨 모히칸 하는 비용 물어봐야겠죠', nan, '모히칸'),
 ('벨라헤어 쉼표머리 액수 물어볼게요', nan, '쉼표머리'),
 ('더아름다움 가일컷 요금 가르쳐주세요', nan, '가일컷'),
 ('도담헤어 그레이스펌 하는 비용 가르쳐주세요', nan, '그레이스펌'),
 ('준오헤어 당산역점 발롱펌 하는 비용 어떤가요?', nan, '발롱펌'),
 ('나의 봄 헤어 히피펌 액수 여쭤볼려고', nan, '히피펌'),
 ('루이스 쉐도우펌 값 여쭤보려고요', nan, '쉐도우펌'),
 ("한's헤어 모히칸 요금 궁금하네요", nan, '모히칸'),
 ('봉헤어 내츄럴펌 궁금하네요', nan, '내츄럴펌'),
 ('리안헤어 군자역점 펌 비용 여쭤볼려고', '펌', nan),
 ('티지헤어 살롱 고덕그라시움점 C컬펌 요금 궁금한데요', nan, 'C컬펌'),
 ('헤어벨리시마 건대점 젤리펌 액수 아시나요', nan, '젤리펌'),
 ('준오헤어 가든준오 용산아이파크몰점 시스루뱅 물어볼게예', nan, '시스루뱅'),
 ('콜라보엑스 호일펌 가격 알려주나요', nan, '호일펌'),
 ('클라스헤어 C컬펌 하는 비용 어때요?', nan, 'C컬

In [712]:
question=[]
hair_1=[]
hair_2=[]
for i,x,y in price_complete_question:
    question.append(i)
    hair_1.append(x)
    hair_2.append(y)

In [713]:
#dataframe화 작업
df_ques_pri = pd.DataFrame({
    'question' : question
    ,'tag_intention' : 'price'
    ,'tag_hair_1' : hair_1
    ,'tag_hair_2' : hair_2
})
df_ques_pri.tail()

,question,tag_intention,tag_hair_1,tag_hair_2
1185223,러블리하우스 방배점 가르마펌 하는 요금 알아보는데요,price,NaN,가르마펌
1185224,헤어by연 글램펌 요금 어떻습니까?,price,NaN,글램펌
1185225,Tu es Belle 포마드펌 비용 어떻지?,price,NaN,포마드펌
1185226,사틴헤어 불광역점 가르마펌 비용 얼마죠?,price,NaN,가르마펌
1185227,현앤현헤어 리젠트컷 가격 물어봐야겠죠,price,NaN,리젠트컷


## 리뷰 관련 문의 질문 셋 만들기

In [714]:
len(que_review_list_dict)

3573

In [715]:
#리뷰 문의용 질문 생성

# 유형 1. 상호명 + 헤어 카테고리1 + 가격문의 어미
# 유형 2. 상호명 + 헤어 카테고리2 + 가격문의 어미

q_about_review = []
hair_category_1=[]
hair_category_2=[]

# 유형 1. 상호명 + 헤어 카테고리1 + 가격문의 어미
for name in df_store:
    storename = name
    for hair_type in df_hair.hair_category1[0].split(','): 
        random_nums = random.sample(range(0,3573), 6) #6개 랜덤 가격관련 문의 질문 
        for i in random_nums:
            long_question = storename + ' ' + hair_type + ' ' + que_review_list_dict[i]
            hair_category_1.append(hair_type)
            hair_category_2.append(np.nan)
            q_about_review.append(long_question)

# 유형 2. 상호명 + 헤어 카테고리2 + 가격문의 어미
for name in df_store:
    storename = name
    for hair_type in df_hair.hair_category2[0].split(','): 
        random_nums = random.sample(range(0,3573), 6) #6개 랜덤 가격관련 문의 질문 
        for i in random_nums:
            long_question = storename + ' ' + hair_type + ' ' + que_review_list_dict[i]
            hair_category_1.append(np.nan)
            hair_category_2.append(hair_type)
            q_about_review.append(long_question)

In [716]:
len(q_about_review)

1185228

In [717]:
review_complete_question = list(zip(q_about_review, hair_category_1, hair_category_2))
review_complete_question[190:200]

[('차홍룸 잠실점 컷 사람들 후기 여쭤보려고요', '컷', nan),
 ('차홍룸 잠실점 컷 거짓없는 평 여쭤볼라고', '컷', nan),
 ('차홍룸 잠실점 펌 실제평가 있으려나', '펌', nan),
 ('차홍룸 잠실점 펌 진짜후기 있는가요', '펌', nan),
 ('차홍룸 잠실점 펌 거짓없는 평가 있습니까', '펌', nan),
 ('차홍룸 잠실점 펌 반응 알려줄수 있나요', '펌', nan),
 ('차홍룸 잠실점 펌 가본사람들 후기 있는갑네요', '펌', nan),
 ('차홍룸 잠실점 펌 광고없는후기 뭐죠', '펌', nan),
 ('에이엠톤 본점 머리 광고없는리뷰 물어볼라는데요', '머리', nan),
 ('에이엠톤 본점 머리 평점 있는지 궁금해요', '머리', nan)]

In [718]:
q_about_review[30500:30510]

['아이브로우바 바이이연 홍대점 컷 사람들 후기 물어볼게요',
 '아이브로우바 바이이연 홍대점 컷 가본사람들 평가 뭔가요',
 '아이브로우바 바이이연 홍대점 컷 진짜반응 있다면서요',
 '아이브로우바 바이이연 홍대점 컷 사람들 후기 뭐고',
 '아이브로우바 바이이연 홍대점 펌 거짓없는 후기 궁금해요',
 '아이브로우바 바이이연 홍대점 펌 실제리뷰 알아',
 '아이브로우바 바이이연 홍대점 펌 평 있어예',
 '아이브로우바 바이이연 홍대점 펌 광고없는후기 알수있나요',
 '아이브로우바 바이이연 홍대점 펌 가본사람들 평 알려주세요',
 '아이브로우바 바이이연 홍대점 펌 진짜반응 있어요']

In [719]:
random.shuffle(review_complete_question)
review_complete_question[1400:1410]

[('탑스타일 내츄럴펌 평가 있나', nan, '내츄럴펌'),
 ('메종드ic 보브컷 거짓없는 리뷰 있는지요', nan, '보브컷'),
 ('리안헤어 영등포신세계점 드레드락 실제평점 있는건가요', nan, '드레드락'),
 ('미립헤어 쉼표머리 가본사람들 평 알려주시겠어요', nan, '쉼표머리'),
 ('준오헤어 신림역2호점 투블럭펌 후기 알려주실래요', nan, '투블럭펌'),
 ('제오헤어 오류점 댄디컷 광고없는후기 있나봐예', nan, '댄디컷'),
 ('지노헤어 명지대점 쉼표머리 가본사람들 반응 뭔가요', nan, '쉼표머리'),
 ('주헤어 리프펌 평점 뭐가있습니까', nan, '리프펌'),
 ('펑크샬롬 디자인펌 사람들 리뷰 있겠죠', nan, '디자인펌'),
 ('어바웃9 신림대세점 붙임머리 솔직리뷰 있나요', nan, '붙임머리')]

In [720]:
question=[]
hair_1=[]
hair_2=[]

for i,x,y in review_complete_question:
    question.append(i)
    hair_1.append(x)
    hair_2.append(y)

In [721]:
#dataframe화 작업
df_ques_review = pd.DataFrame({
    'question' : question
    ,'tag_intention' : 'review'
    ,'tag_hair_1' : hair_1
    ,'tag_hair_2' : hair_2
})
df_ques_review[850:860]

,question,tag_intention,tag_hair_1,tag_hair_2
850,로이드밤 금천구청역점 허쉬컷 진짜평가 있으세요,review,NaN,허쉬컷
851,로이드밤 우장산역점 볼륨매직펌 평 알려줄래,review,NaN,볼륨매직펌
852,보떼헤어 모히칸 가본사람들 후기 뭔고,review,NaN,모히칸
853,헤어스케치 페미닌펌 실제리뷰 알려줄래,review,NaN,페미닌펌
854,바나나미용실 바디펌 진짜평가 뭐가 있습니까,review,NaN,바디펌
855,살롱드 비쥬벨헤어 프릴펌 실제평가 알려줄수 있나요,review,NaN,프릴펌
856,뜨뜨 헤어살롱 호일펌 진실된 평가 알아,review,NaN,호일펌
857,살롱드픽스 사가정점 픽시컷 가본사람들 리뷰 있지,review,NaN,픽시컷
858,리프레쉬헤어 러블리펌 평점 찾아주세요,review,NaN,러블리펌
859,호야즈룸 보울컷 평 있는건가요,review,NaN,보울컷


In [722]:
req_question[:10]

,question,tag_intention,tag_adjs,tag_hair_1,tag_hair_2
682179,실력있는 셋팅펌 추천할만한 헤어샵 물어볼게요,recommendation,satisfaction,NaN,셋팅펌
4004,꼼꼼하게 챙겨주는 머리 실패하지않는 곳 뭐뭐있어요,recommendation,customer_service,머리,NaN
585,비위생적이지 않은 머리 하고싶은 곳 부탁해,recommendation,sanitation,머리,NaN
545335,남심 저격할만한 트위스트펌 솜씨좋은 헤어샵 어디에요,recommendation,satisfaction,NaN,트위스트펌
339263,가격이 개이득인 S컬펌 솜씨좋은 데 아시나요,recommendation,price,NaN,S컬펌
1082269,대만족의 하고싶은 데 알려주실래요,recommendation,satisfaction,NaN,NaN
485508,걸어서 5분 거리의 투블럭 쩌는 미용실 있습니까,recommendation,location,NaN,투블럭
364937,값 싼 루즈펌 알려주고 싶은 헤어샵 뭔데에,recommendation,price,NaN,루즈펌
899145,헤어스타일 변신하기 좋은 내츄럴펌 추천해줄만한 헤어샵 여쭤볼려고,recommendation,etc,NaN,내츄럴펌
956334,여름맞이용 보브컷 실패하지않는 헤어샵 뭡니까,recommendation,etc,NaN,보브컷


In [723]:
# 지금까지 생성한 질문 말뭉치를 모두 통합하기(추천/문의)
df_all_question = pd.concat([req_question, df_ques_loc, df_ques_pri, df_ques_review]
                            , sort = True
                           )
df_all_question = df_all_question[['question','tag_intention','tag_adjs','tag_hair_1','tag_hair_2']]
print(df_all_question.shape)
df_all_question[98540:98550]

(4626576, 5)


,question,tag_intention,tag_adjs,tag_hair_1,tag_hair_2
305691,내부가 화려한 S컬펌 예쁘게 하는 곳 알려줄래,recommendation,mood,NaN,S컬펌
9825,인테리어가 화려한 컷 잘해주는 데 알려주시겠어요,recommendation,mood,컷,NaN
957651,여름맞이용 드레드락 알려주고 싶은 곳 알아보는데요,recommendation,etc,NaN,드레드락
793403,머리 관리 잘해주는 가르마펌 하고싶은 미용실 여쭤보려구요,recommendation,satisfaction,NaN,가르마펌
706451,금손인 디자이너가 많은 리프컷 잘해주는 미용실 뭐가 있습니까,recommendation,satisfaction,NaN,리프컷
39861,대학생 사이에서 인기많은 컷 추천해줄만한 데 여쭤보려하는데요,recommendation,etc,컷,NaN
314856,분위기가 좋은 애즈펌 실패한적 없는 미용실 어디지,recommendation,mood,NaN,애즈펌
308538,내부가 화려한 스핀스왈로펌 실패한적 없는 미용실 있었나요,recommendation,mood,NaN,스핀스왈로펌
834279,단정해보이는 레이어드컷 예쁘게 하는 헤어샵 뭡니까,recommendation,satisfaction,NaN,레이어드컷
480058,접근성이 좋은 픽시컷 추천해줄만한 헤어샵 있는가,recommendation,location,NaN,픽시컷


In [724]:
#질문들을 랜덤하게 셔플한다
from sklearn.utils import shuffle
df_all_question = shuffle(df_all_question)

In [725]:
#인덱스 재설정
df_all_question.reset_index(drop=True, inplace=True)

In [726]:
df_all_question[900:910] #질문 모두 잘 섞임

,question,tag_intention,tag_adjs,tag_hair_1,tag_hair_2
900,신촌동에 있는 헤어샵 어디,location,NaN,NaN,NaN
901,금손인 쌤이 많은 트위스트펌 걱정없는 데 알려줘요,recommendation,satisfaction,NaN,트위스트펌
902,공릉1동에 있는 헤어샵 상세주소 찾아줘요,location,NaN,NaN,NaN
903,제오헤어 마곡나루역점 붙임머리 하는 가격 알려줘요,price,NaN,NaN,붙임머리
904,대치동에 있는 미장원 자세한 위치 물어봐도 될까,location,NaN,NaN,NaN
905,마포구에 있는 미용실 상세주소 알려줘,location,NaN,NaN,NaN
906,카미유헤어 매직펌 값 여쭤볼려고,price,NaN,NaN,매직펌
907,아기자기한 댄디컷 머리 맛집 알려줘요,recommendation,mood,NaN,댄디컷
908,벨루스헤어 보니펌 진짜반응 뭐뭐있어요,review,NaN,NaN,보니펌
909,살롱누보 자세한 주소 어디에있어요,location,NaN,NaN,NaN


In [727]:
df_all_question.shape #대략 450만개 질문셋 완성

(4626576, 5)

In [728]:
df_all_question.to_csv('./all_random_questions.csv', encoding='utf-8-sig')

# 완성된 문장 Mecab과 stopwords 적용하여 형태소 처리 하기

In [729]:
korean_re = re.compile('[^ ㄱ-ㅣ가-힣]+') 

def question_tokenize(text):
        #stopwords 설정 - 한국어 불용어 txt 파일 사용
        s_file_name = open('./한국어 불용어(stop_words)정리.txt','r', encoding='utf-8')
        stop_words_list = []
        for line in s_file_name.readlines():
            stop_words_list.append(line[:-1])
        stop_words = stop_words_list
        
        #mecab tokenize
        mecab = Mecab()
        
        #mecab으로 text를 형태소 단위로 나눈뒤, 불용어 처리 후 한글인 것만 리스트에 저장
        text = korean_re.sub('',text)
        text = ' '.join(token for token in mecab.morphs(text)if token not in stop_words)  
        
        return text

In [ ]:
%%time 
#약 450만개의 질문 셋 stopwords 처리 및 형태소 변환 시 걸리는 시간 2시간반임.
all_question_mecab = df_all_question.question.apply(question_tokenize)

In [91]:
#all_question_mecab= 형태소로 나눈 데이터
df_all_question.insert(1, 'question_mecab', all_question_mecab)

In [92]:
df_all_question.tail()

,question,question_mecab,tag_intention,tag_adjs,tag_hair
4567387,아뜰리에7 스핀스왈로펌 비용 어떻지?,뜰 리 스핀 스 왈 로펌 비용 어떻,price,NaN,스핀스왈로펌
4567388,체인점이 많은 붙임머리 예쁘게 하는 곳 어디인지 알까요,체인점 많 붙임 머리 예쁘 게 곳 인지 알 까요,recommendation,satisfaction,붙임머리
4567389,살롱드미미 스핀스왈로우 펌 후기 있는가,살 롱드 미미 스핀 스왈로우 펌 후기 있 는가,review,NaN,스핀스왈로우 펌
4567390,불편하지 않은 베이비펌 솜씨좋은 헤어샵 궁금해서요,불편 않 베이비펌 솜씨 좋 헤어 샵 궁금 요,recommendation,mood,베이비펌
4567391,박준뷰티랩 성균관대점 페미닌펌 질문합니다,박준 뷰티 랩 성균관 대점 페미 닌 펌 질문,price,NaN,페미닌펌


In [93]:
df_all_question.to_csv('./all_question_by_mecab.csv', encoding='utf-8-sig')

## TF-IDF 벡터화

In [110]:
df_all_question[100:150]

,question,question_mecab,tag_intention,tag_adjs,tag_hair
100,ONL HAIR 머쉬룸컷 솔직후기 있는건가요,머쉬룸 컷 솔 직후 기 있,review,NaN,머쉬룸컷
101,일부러 찾아갈만한 물결펌 추천해줄만한 헤어샵 어디냐,일부러 찾아갈 만 물결 펌 추천 줄 만 헤어 샵 냐,recommendation,satisfaction,물결펌
102,걸어서 3분 이내의 볼륨매직 훌륭한 데 뭐가있죠,걸어서 분 이내 볼륨 매직 훌륭 데 뭐 있 죠,recommendation,location,볼륨매직
103,상담 꼼꼼하게 해주는 포마드컷 잘해주는 미용실 알아,상담 꼼꼼 게 주 포마드 컷 잘 주 미용실 알,recommendation,satisfaction,포마드컷
104,메이블랑 헤어 상세주소 알려줘요,메이블 랑 헤어 상세 주소 알려 줘요,location,NaN,NaN
105,아이디헤어 롯데백화점 청량리점 머쉬룸컷 추가 가격 어떻습니까?,아이디 헤어 롯데 백화점 청량리 점 머쉬룸 컷 추가 가격 어떻,price,NaN,머쉬룸컷
106,프롬제이헤어살롱 장소 아냐 모르냐,프롬 제이 헤어 살롱 장소 아냐 모르 냐,location,NaN,NaN
107,수미살롱 베이비펌 하는 비용 얼마죠?,수미 살롱 베이비펌 비용 죠,price,NaN,베이비펌
108,인헤어 드레드락 거짓없는 리뷰 뭐가있죠,헤어 드레드 락 거짓 없 리뷰 뭐 있 죠,review,NaN,드레드락
109,리안헤어 뚝섬유원지역점 자세한 장소 어디지,리안 헤어 뚝섬유원지역 점 자세 장소,location,NaN,NaN


In [111]:
def sum_tags(df):
    list_for_tag = []
    
    intention = df['tag_intention']
    feature = df['tag_adjs']
    category = df['tag_hair']
    
    if intention is not np.nan:
        list_for_tag.append(intention)
    if feature is not np.nan:
        list_for_tag.append(feature)
    if category is not np.nan:
        list_for_tag.append(category)
    
    return list_for_tag

In [112]:
%%time
df_all_question['all_tags'] = df_all_question.apply(sum_tags, axis=1) 

Wall time: 2min 12s


In [116]:
df_all_question[160:170]

,question,question_mecab,tag_intention,tag_adjs,tag_hair,all_tags
160,수인헤어 쉼표머리 광고없는리뷰 뭔가요,수인 헤어 쉼표 머리 광고 없 리뷰 뭔가요,review,NaN,쉼표머리,"[review, 쉼표머리]"
161,레드썬 바디펌 실제평가 물어볼려고요,레드 썬 바디 펌 실제 평가 물 볼 려고요,review,NaN,바디펌,"[review, 바디펌]"
162,남친이랑 같이 오고 싶은 울프컷 오지는 헤어샵 질문합니다,남친 이랑 싶 울프 컷 오지 헤어 샵 질문,recommendation,satisfaction,울프컷,"[recommendation, satisfaction, 울프컷]"
163,갸름해보이는 가르마펌 최고인 데 아세요,갸름 보이 가르마 펌 최고 데 아세요,recommendation,satisfaction,가르마펌,"[recommendation, satisfaction, 가르마펌]"
164,설헤어살롱 가르마펌 액수 가르쳐주세요,설 헤어 살롱 가르마 펌 액수 가르쳐 주 세요,price,NaN,가르마펌,"[price, 가르마펌]"
165,매일 청소하는 볼륨매직펌 지리는 헤어샵 뭐가 있습니까,매일 청소 볼륨 매직 펌 지리 헤어 샵 뭐 있,recommendation,sanitation,볼륨매직펌,"[recommendation, sanitation, 볼륨매직펌]"
166,오다헤어 이수점 히메컷 추가 가격 물어봐야겠죠,오다 헤어 이수점 히메컷 추가 가격 물어봐야 겠죠,price,NaN,히메컷,"[price, 히메컷]"
167,준오헤어 강남2호점 스핀스왈로펌 진짜평가 궁금해요,준 오헤어 강남호 점 스핀 스 왈 로펌 진짜 평가 궁금,review,NaN,스핀스왈로펌,"[review, 스핀스왈로펌]"
168,my name is 영웅. 헤어&탈모 볼륨매직펌 하는 비용 부탁드려요,영웅 헤어 탈모 볼륨 매직 펌 비용 부탁,price,NaN,볼륨매직펌,"[price, 볼륨매직펌]"
169,아가페헤어 히메컷 비용 아세요,아가페 헤어 히메컷 비용 아세요,price,NaN,히메컷,"[price, 히메컷]"


In [114]:
df_all_question.to_csv('./all_question_by_mecab_tag.csv', encoding='utf-8-sig')

In [115]:
pd.read_csv('./all_question_by_mecab_tag.csv', encoding='utf-8-sig',index_col=0)

C:\Users\rainb\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,question,question_mecab,tag_intention,tag_adjs,tag_hair,all_tags
0,헤어BS12 리프컷 별도 액수 알수있나요,헤어 리프 컷 별도 액수 알 있 나요,price,NaN,리프컷,"['price', '리프컷']"
1,헤어더뷰 성신여대점 스핀스왈로펌 추가 가격 궁금하네요,헤어 더 뷰 성신 대점 스핀 스 왈 로펌 추가 가격 궁금,price,NaN,스핀스왈로펌,"['price', '스핀스왈로펌']"
2,밝은 러블리펌 솜씨좋은 미용실 뭐죠,밝 러블리 펌 솜씨 좋 미용실 뭐 죠,recommendation,mood,러블리펌,"['recommendation', 'mood', '러블리펌']"
3,보그헤어 잠실새내역점 자세한 위치 어디인지 말해줘요,보그 헤어 잠실 새내 역점 자세 위치 인지 말 해줘요,location,NaN,NaN,['location']
4,균형잡힌 웨이브펌 훌륭한 데 뭔데에,균형 잡힌 웨이브 펌 훌륭 데 뭔 데,recommendation,satisfaction,웨이브펌,"['recommendation', 'satisfaction', '웨이브펌']"
...,...,...,...,...,...,...
4567387,아뜰리에7 스핀스왈로펌 비용 어떻지?,뜰 리 스핀 스 왈 로펌 비용 어떻,price,NaN,스핀스왈로펌,"['price', '스핀스왈로펌']"
4567388,체인점이 많은 붙임머리 예쁘게 하는 곳 어디인지 알까요,체인점 많 붙임 머리 예쁘 게 곳 인지 알 까요,recommendation,satisfaction,붙임머리,"['recommendation', 'satisfaction', '붙임머리']"
4567389,살롱드미미 스핀스왈로우 펌 후기 있는가,살 롱드 미미 스핀 스왈로우 펌 후기 있 는가,review,NaN,스핀스왈로우 펌,"['review', '스핀스왈로우 펌']"
4567390,불편하지 않은 베이비펌 솜씨좋은 헤어샵 궁금해서요,불편 않 베이비펌 솜씨 좋 헤어 샵 궁금 요,recommendation,mood,베이비펌,"['recommendation', 'mood', '베이비펌']"
